In [1]:
from autofeat.dataset import *

source_dataset = KaggleDataset(id="mahmoudemadabdallah/hr-analytics-employee-attrition-and-performance")

In [2]:
from autofeat.transform import *

transform = (
    Cast()
    .then(Encode())
    .then(Aggregate(by={"EmployeeID"}))
    .then(Identity(), Join(on={"EmployeeID"}))
    .then(Identity(), Combine())
)

In [3]:
derived_dataset = source_dataset.derive(transform)

In [4]:
features = derived_dataset.features(
    Filter(eq={"EmployeeID": "51A4-EA6E"}),
    Filter(eq={"EmployeeID": "A4A4-F49C"}),
)

In [5]:
features.collect()

count(*) from RatingLevel.csv,max(RatingID) from RatingLevel.csv,mean(RatingID) from RatingLevel.csv,median(RatingID) from RatingLevel.csv,min(RatingID) from RatingLevel.csv,std(RatingID) from RatingLevel.csv,sum(RatingID) from RatingLevel.csv,var(RatingID) from RatingLevel.csv,count(*) from PerformanceRating.csv,max(ManagerRating) from PerformanceRating.csv,mean(ManagerRating) from PerformanceRating.csv,median(ManagerRating) from PerformanceRating.csv,min(ManagerRating) from PerformanceRating.csv,std(ManagerRating) from PerformanceRating.csv,sum(ManagerRating) from PerformanceRating.csv,var(ManagerRating) from PerformanceRating.csv,max(JobSatisfaction) from PerformanceRating.csv,mean(JobSatisfaction) from PerformanceRating.csv,median(JobSatisfaction) from PerformanceRating.csv,min(JobSatisfaction) from PerformanceRating.csv,std(JobSatisfaction) from PerformanceRating.csv,sum(JobSatisfaction) from PerformanceRating.csv,var(JobSatisfaction) from PerformanceRating.csv,max(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,mean(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,median(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,min(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,std(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,sum(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,var(TrainingOpportunitiesWithinYear) from PerformanceRating.csv,max(TrainingOpportunitiesTaken) from PerformanceRating.csv,mean(TrainingOpportunitiesTaken) from PerformanceRating.csv,median(TrainingOpportunitiesTaken) from PerformanceRating.csv,min(TrainingOpportunitiesTaken) from PerformanceRating.csv,std(TrainingOpportunitiesTaken) from PerformanceRating.csv,sum(TrainingOpportunitiesTaken) from PerformanceRating.csv,var(TrainingOpportunitiesTaken) from PerformanceRating.csv,…,max(YearsWithCurrManager) / count(*) from Employee.csv,min(YearsInMostRecentRole) + std(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) - std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * std(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) / std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) / min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) + min(Education) from Employee.csv,min(YearsInMostRecentRole) - min(Education) from Employee.csv,min(Education) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * min(Education) from Employee.csv,min(YearsInMostRecentRole) / min(Education) from Employee.csv,min(Education) / min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) + max(YearsWithCurrManager) from Employee.csv,min(YearsInMostRecentRole) - max(YearsWithCurrManager) from Employee.csv,max(YearsWithCurrManager) - min(YearsInMostRecentRole) from Employee.csv,min(YearsInMostRecentRole) * max(YearsWithCurrManager) from Employee.csv,min(YearsInMostRecentRole) / max(YearsWithCurrManager) from Employee.csv,max(YearsWithCurrManager) / min(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) + min(Education) from Employee.csv,std(YearsInMostRecentRole) - min(Education) from Employee.csv,min(Education) - std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) * min(Education) from Employee.csv,std(YearsInMostRecentRole) / min(Education) from Employee.csv,min(Education) / std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) + max(YearsWithCurrManager) from Employee.csv,std(YearsInMostRecentRole) - max(YearsWithCurrManager) from Employee.csv,max(YearsWithCurrManager) - std(YearsInMostRecentRole) from Employee.csv,std(YearsInMostRecentRole) * max(YearsWithCurrManager) from Employee.csv,std(YearsInMostRecentRole) / max(YearsWithCurrManager) from Employee.csv,max(YearsWithCurrManager) / std(YearsInMostRecentRole) from Employee.csv,min(Education) + max(YearsWit

In [6]:
import numpy
import sklearn.feature_selection
import xgboost

examples = [
    (
        Filter(eq={"EmployeeID": "51A4-EA6E"}),
        True,
    ),
    (
        Filter(eq={"EmployeeID": "A4A4-F49C"}),
        False,
    ),
]

features = (
    derived_dataset
    .features(filter for filter, _ in examples)
    .collect()
)

selector = sklearn.feature_selection.SelectFromModel(
    xgboost.XGBClassifier(missing=float("inf")),
    max_features=10,
)

selector.fit(
    X=features.to_numpy(),
    y=numpy.array([label for _, label in examples]),
)

selection = [
    features.columns[i]
    for i, selected in enumerate(selector.get_support())
    if selected
]

In [7]:
selection


['count(*) from RatingLevel.csv',
 'max(RatingID) from RatingLevel.csv',
 'mean(RatingID) from RatingLevel.csv',
 'median(RatingID) from RatingLevel.csv',
 'min(RatingID) from RatingLevel.csv',
 'std(RatingID) from RatingLevel.csv',
 'sum(RatingID) from RatingLevel.csv',
 'var(RatingID) from RatingLevel.csv',
 'count(*) from PerformanceRating.csv',
 'max(ManagerRating) from PerformanceRating.csv']